In [2]:
import pandas as pd
from bs4 import BeautifulSoup as soup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [3]:
#configuring splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\rspic\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


NASA Mars News

In [4]:
#Opening the NASA site in the browser
NASA_url = 'https://redplanetscience.com/'
browser.visit(NASA_url)

In [5]:
#Creating a beautiful soup object
html = browser.html
NASA_soup = soup(html, 'html.parser')

In [6]:
#storing the first title text in news_title and title paragraph in news_p
news_title = NASA_soup.find('div', class_='content_title').get_text()
news_p = NASA_soup.find('div', class_='article_teaser_body').get_text()

JPL Mars Space Images - Featured Image

In [7]:
#Navigate to the spave JPL url
JPL_url = 'https://spaceimages-mars.com'
browser.visit(JPL_url)

In [8]:
#Create a beautiful soup object
html = browser.html
mars_soup = soup(html, 'html.parser')

In [9]:
#Storing the image relative path in mars_url
mars_url = mars_soup.find('img', class_='headerimage')['src']

#Using relative img url to create absolute path
mars_img_url = f'{JPL_url}/{mars_url}'

In [10]:
mars_img_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

Mars Facts

In [11]:
mars_facts_url = 'https://galaxyfacts-mars.com'
mars_facts_df = pd.read_html(mars_facts_url)[0]

In [12]:
mars_facts_df.columns=['Description', 'Mars', 'Earth']
mars_facts_df = mars_facts_df.iloc[1:]
mars_facts_df = mars_facts_df.reset_index(drop=True)
mars_facts_df.set_index('Description', inplace=True)

In [13]:
mars_facts_df

,Mars,Earth
Description,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [14]:
#converting mars df to html table
mars_table =  mars_facts_df.to_html(classes=["table", 'table-striped'])
#Stripping unwanted newlines 
stripped_html_table = mars_table.replace('\n', '')

Mars Hemispheres

In [15]:
astrogeology_url = 'https://marshemispheres.com/'

browser.visit(astrogeology_url)

In [16]:
html = browser.html
hemisphere_soup = soup(html, "html.parser")

In [17]:
#Create a list of all of the hemisphere images
hemispheres = hemisphere_soup.find_all('div', class_='description')

In [18]:
#Create a list to hold the image links.
hemisphere_images = []

for div in hemispheres:
    link = div.find('a', class_='itemLink')['href']
    img_link = f'{astrogeology_url}{link}'

    #append list image urls
    hemisphere_images.append(img_link)

In [19]:
hemisphere_image_urls = []

#loop through the list of image urls
for url in hemisphere_images:
    #visit each url in the images url list
    browser.visit(url)
    #Create beautifulsoup object
    html = browser.html
    img_soup = soup(html, "html.parser")

    #parse to find the image url
    hemisphere_url = img_soup.find_all('a', target='_blank')[2]['href']
    img_url = f'{astrogeology_url}{hemisphere_url}'

    img_title = img_soup.find('h2',class_='title').text
    
    hemisphere_dict = {
        "title": img_title,
        "img_url": img_url
    }

    hemisphere_image_urls.append(hemisphere_dict)

In [20]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]

In [21]:
browser.quit()